In [1]:
import augur
from augur.piper_reader import PiperMail
from sqlalchemy.ext.declarative import declarative_base
import json
import pandas as pd
from perceval.backends.core.pipermail import Pipermail,PipermailList
from perceval.backends.core.mbox import MBox
import perceval
import os, os.path
import sqlalchemy as s
from sqlalchemy.orm import scoped_session, sessionmaker
import datetime
import numpy as np
from dateutil.parser import parse

## Load:
First loads the users data from 'augur.config.json' so will take the Database information (e.g. name of database, port of database). Then connects to the database using augur.App.ghtorrentplus() and also loads the piper_reader and loads the path to the list of mailing lists 'runtine/mailing_lists.csv'

In [2]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect = augurApp.ghtorrentplus()
Piper, path= augurApp.piper()

## Connect:

Queries what tables are in the database and determines if 'mail_lists' is there if it is 'mail_lists' is set as 'True' if it isn't 'mail_lists' is set at 'False'. Then we determine what mailing lists are in 'mailing_list_jobs' and we determine how many rows are in it. If 'mailing_list_jobs' is not in the Database it is created and the column 'augurlistID' is set as the primary key. We then add a connection to 'mailing_list_jobs' so that we can change the column 'last_message_date' if new messages were downloaded for a mailing list.

In [4]:
table_names = s.inspect(connect.db).get_table_names()
print(table_names)
val = False
mail_lists = True
numb = 0
if("mail_lists" not in table_names):
    mail_lists = False
if("mailing_list_jobs" in table_names):
    lists_createdSQL = s.sql.text("""SELECT project FROM mailing_list_jobs""")
    df1 = pd.read_sql(lists_createdSQL, connect.db)
    print(df1)
    val = True
    val = connect.db.engine.execute("""SELECT augurlistID FROM 
                                   mailing_list_jobs
                                   ORDER BY augurlistID DESC LIMIT 1""")
    for i in val:
        numb = i['augurlistID']
    
else:
    columns2 = "augurlistID","backend_name","mailing_list_url","project","last_message_date"
    df_mail_list = pd.DataFrame(columns=columns2)
    df_mail_list.to_sql(name="mailing_list_jobs",con=connect.db,if_exists='replace',index=False,
                        dtype={'augurlistID': s.types.Integer(),
                               'backend_name': s.types.VARCHAR(length=300),
                               'mailing_list_url': s.types.VARCHAR(length=300),
                               'project': s.types.VARCHAR(length=300),
                               'last_message_date': s.types.DateTime()
                        })
    lists_createdSQL = s.sql.text("""SELECT project FROM mailing_list_jobs""")
    df1 = pd.read_sql(lists_createdSQL, connect.db)
    connect.db.execute("ALTER TABLE mailing_list_jobs ADD PRIMARY KEY (augurlistID)")
    print("Created Table")
Base = declarative_base(connect.db)
class table(Base):
    __tablename__ = 'mailing_list_jobs'
    __table_args__={'autoload':True}
    
Session = sessionmaker(bind=connect.db)
session = Session()
res = session.query(table).all()

['github_issues', 'github_issues_sentiment_scores', 'github_pull_requests', 'github_pull_requests_sentiment_scores', 'github_repo_jobs', 'issue_response_time']
Created Table


## Sifting through messages:

The function 'write_message' is used to fetch the messages from the downloaded MBox's. The messages downloaded can either be from the last message was downloaded when the jupyter notebook was downloaded previously or if it's the first time all the messages are downloaded. The function 'write_message' then determines which message contains the full message thread by looking to see if the 'ID' for the message is stored in 'References'. Only if the next message downloaded does not reference the previous message will the message be added to dictionary and if a certain amount of messages are stored it is added to the table 'mail_lists' using piper_reader.

In [5]:
def write_message(repo,type_archive,mail_check,pos,db,res,session,archives,numb,mail_lists,time=None):
    thread = None
    store = None
    k = 0
    di = {}
    for message in repo.fetch(from_date=time):
        if(type_archive == 'not_new'):
            mess_check = Piper.convert_date(message['data']['Date'])
        if(type_archive == 'not_new' and mess_check <= time ):
            continue            
        elif(type_archive == 'not_new' and mess_check > time):
            mail_check[pos] = 'update'
            
        ID = message['data']['Message-ID']
        try:
            message['data']['References']                           
            if((not thread == None) and (thread['data']['Message-ID'] not in message['data']['References'])):
                di[k] = thread
                store = None
                k+=1
                
            elif( (not store == None) and (store['data']['Message-ID'] not in message['data']['References'])):
                di[k] = store
                store = None
                k+=1
            thread = message
        except:
            if(not thread == None):
                di[k] = thread
                thread = None
                k+=1
            elif(not store == None):
                di[k] = store
                store = None
                k+=1
            store = message
        if(len(di) == 5000):
            numb,mail_lists = Piper.make(connect.db,mail_check,archives,mail_lists,res,session,di,numb)
            di = {}
            k = 0
    if(len(di) < 5000 and len(di) > 0):
        print(len(di))
        numb,mail_lists = Piper.make(connect.db,mail_check,archives,mail_lists,res,session,di,numb)
        di = {}
        k = 0
    else:
        di = {}
        k = 0
    if( (thread == None) and (store == None)):
        good = 1
    elif( (thread == None) and (not store == None) ):
        di[k] = store
    elif( (store == None) and (not thread == None)):
        di[k] = thread
    elif(store['data']['Message-ID'] in thread['data']['References']):
        di[k] = thread
    else:
        di[k] = store
    if(bool(di)):
        numb,mail_lists = Piper.make(connect.db,mail_check,archives,mail_lists,res,session,di,numb)
    return numb,mail_lists
    

## Iteration through mailing lists:

Determines if the file with the mailing lists was created, if not it writes a set of default mailing lists (to show how the program would work). The mailing lists are then loaded into a dataframe and we iterate through the mailing lists by grouping them by the links. We then check to see if the mailing list is in the 'mailing_list_jobs' table in the SQL Database and if so we assign 'not_new' to 'mail_check' and store the last message date that is stored in 'mailing_list_jobs' to 'time'. If the mailing list is not in 'mailing_list_jobs' we assign 'new' to 'mail_check' and the date is set to 'None' for 'time'.

In [6]:
if(not os.path.exists(path)):
    file = open(path, "w+")
else:
    file = open(path, "r")
    print("yeah")
if (os.stat(path).st_size == 0):
    file.write("Link,mail_list\n")
    file.write("https://lists.opendaylight.org/pipermail/,\"aalldp-dev\"\n")
    file.write("https://lists.opendaylight.org/pipermail/,\"archetypes-dev\"\n")
    print("Please enter the mailing lists and the links for them please")
    print("Going to the default mailing lists")

count = 0
for line in file:
    print(line)
    count+=1
    if(count == 2):
        break
if(count == 2):
    file.close()
    df = pd.read_csv(path)
    groups = df.groupby('Link').groups
    for group in groups:
        link = group
        mail_list = (df.loc[df['Link'] == group]['mail_list']).tolist()
        print(mail_list,"mail_list")            
        mail_check = {key:False for key in mail_list}
        print(mail_check)
        file = "mail_list"
        path = "/augur/data/archive-" 
        #numb = 0
        for x in range(len(mail_list)):
            #print(link+mail[x])
            if(mail_list[x] not in df1['project'].values ):
                mail_check[mail_list[x]] = 'new'
                place = os.path.join(os.getcwd() + path + mail_list[x] +'.json')           
                repo = Pipermail(url = link+ mail_list[x] + "/",dirpath="tmp/archives_"+mail_list[x])
                numb,mail_lists = write_message(repo,'new',mail_check,mail_list[x],connect.db,res,session,\
                                     [mail_list[x]],numb,mail_lists)
                print("Created File",mail_list[x])
            else:
                last_updatedSQL = s.sql.text("""SELECT last_message_date FROM 
                mailing_list_jobs WHERE project = """ +  "'" + mail_list[x] + "'")
                last_updated_df = pd.read_sql(last_updatedSQL, connect.db)
                time = (last_updated_df['last_message_date'])  
                time = time.astype(object)
                place = os.path.join(os.getcwd() + path + 'temp_' + mail_list[x] +'.json')       
                repo = Pipermail(url = link+ mail_list[x] + "/",dirpath="tmp/archives_"+mail_list[x])
                print(time[0])
                print(type(time[0]))
                numb,mail_lists = write_message(repo,'not_new',mail_check,mail_list[x],connect.db,\
                              res,session,[mail_list[x]],numb,mail_lists,time[0])
                print("Checking to see for updated messages")
        print(mail_check)
        print("Finished downloading files")

2018-08-01 16:50:02 keanu-Inspiron-5567 perceval.backends.core.pipermail[18033] INFO Looking for messages from 'https://lists.opendaylight.org/pipermail/aalldp-dev/' since 1970-01-01 00:00:00+00:00
2018-08-01 16:50:02 keanu-Inspiron-5567 perceval.backends.core.pipermail[18033] INFO Downloading mboxes from 'https://lists.opendaylight.org/pipermail/aalldp-dev/' to since 1970-01-01 00:00:00+00:00


runtime/mailing_lists.csv Place
yeah
Link,mail_list

https://lists.opendaylight.org/pipermail/,"aalldp-dev"

['aalldp-dev', 'archetypes-dev'] mail_list
{'aalldp-dev': False, 'archetypes-dev': False}


2018-08-01 16:50:07 keanu-Inspiron-5567 perceval.backends.core.pipermail[18033] INFO 2/2 MBoxes downloaded
2018-08-01 16:50:07 keanu-Inspiron-5567 perceval.backends.core.mbox[18033] INFO Done. 6/6 messages fetched; 0 ignored
2018-08-01 16:50:07 keanu-Inspiron-5567 perceval.backends.core.pipermail[18033] INFO Fetch process completed


getting
getting
yep
got-em
4
['aalldp-dev']
2018-07-06 18:39:58
File uploaded  13
Mailing List Job uploaded
Finished
['aalldp-dev']
File uploaded  4


2018-08-01 16:50:31 keanu-Inspiron-5567 perceval.backends.core.pipermail[18033] INFO Looking for messages from 'https://lists.opendaylight.org/pipermail/archetypes-dev/' since 1970-01-01 00:00:00+00:00
2018-08-01 16:50:31 keanu-Inspiron-5567 perceval.backends.core.pipermail[18033] INFO Downloading mboxes from 'https://lists.opendaylight.org/pipermail/archetypes-dev/' to since 1970-01-01 00:00:00+00:00


Mailing List Job uploaded
Finished
Created File aalldp-dev


2018-08-01 16:50:33 keanu-Inspiron-5567 perceval.backends.core.pipermail[18033] INFO 1/1 MBoxes downloaded
2018-08-01 16:50:33 keanu-Inspiron-5567 perceval.backends.core.mbox[18033] INFO Done. 2/2 messages fetched; 0 ignored
2018-08-01 16:50:33 keanu-Inspiron-5567 perceval.backends.core.pipermail[18033] INFO Fetch process completed


getting
1
['archetypes-dev']
File uploaded  4
Mailing List Job uploaded
Finished
['archetypes-dev']
File uploaded  4
Mailing List Job uploaded
Finished
Created File archetypes-dev
{'aalldp-dev': 'new', 'archetypes-dev': 'new'}
Finished downloading files
